In [8]:
import os
import zipfile
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames
from wordToHtmlConvertor.wordToHtmlConvertor import WordToHtmlConvertor


class FolderNotFoundError(Exception):
    pass


def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog):

    module_path = os.path.join(basePath)

    if "/" in basePath:
        pathSep = "/"
    else:
        pathSep = "\\"
    
    # Generate output folder path
    output_json_path = os.path.join(basePath, 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, htmlDocName))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', htmlDocName,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                             controlBasePath=controlBasePath,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType
                                             )

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = os.path.join(output_json_path, htmlDocName)
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                        controlBasePath=controlBasePath,
                                        language=languageCode,
                                        fileName=fileNameQrd,
                                        domain=domain,
                                        procedureType=procedureType
                                        )
    
    path_json = os.path.join(basePath,'outputJSON', fileNameJson)
    print("PathJson",path_json)
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(controlBasePath,
                                basePath,
                                domain,
                                procedureType,
                                languageCode,
                                documentNumber,
                                fileNameDoc,
                                fileNameQrd,
                                fileNameMatchRuleBook,
                                fileNameDocumentTypeNames,
                                fileNameLog,
                                stopWordFilterLen=6,
                                isPackageLeaflet=False,
                                medName=None
                                ):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}_{getRandomString(1)}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        controlBasePath=controlBasePath,
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber
        ).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        controlBasePath,
        basePath,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName)
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll


def parseDocument(controlBasePath, basePath ,htmlDocName, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = None):
    
    if "/" in basePath:
        pathSep = "/"        
    else:
        pathSep = "\\"
    
    fileNameLog = os.path.join(basePath,'FinalLog.txt')

    pathComponents = basePath.split(pathSep)
    print(pathComponents, htmlDocName)
    timestamp = pathComponents[-1]
    languageCode =  pathComponents[-2]
    medName = pathComponents[-3]
    procedureType = pathComponents[-4]
    domain = pathComponents[-5]

    print(timestamp, languageCode, medName, procedureType, domain)
        
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)

    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog)
    
    print("stylePath:-",stylesFilePath)
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(controlBasePath,
                                    basePath,
                                    domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, basePath, coll)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, controlBasePath, basePath, pms_oms_annotation_data, stylesFilePath, medName)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, basePath, generatedXml)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")      

        #return df,coll,dfExtractedHierRR
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")


In [ ]:
wordToHtmlConvertorObj = WordToHtmlConvertor()
wordToHtmlConvertorObj.convertWordToHTML()

In [10]:
# inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
inputZipFolderPath = os.path.abspath(os.path.join('..'))
inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
inputZipFileName = "Zynteglo~H~CAP~en~2021-05-19T07-52-54Z.zip"

In [16]:
fileNameQrd = 'qrd_canonical_model.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'
fsMountName = '/mounted'

info = inputZipFileName.split("~")

try:
    medName = info[0]
    domain = info[1]
    procedureType = info[2]
    languageCode = info[3]
    timestamp = info[4]
    timestamp = timestamp.replace(".zip","")

except Exception:
    raise f"Missing required info in the zip file name {inputZipFileName}"

if "\\" in os.getcwd():
    localEnv = True
    inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
    outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
    controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
else:
    localEnv = False
    inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
    outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
    controlFolderPath = os.path.join(f'{fsMountName}','control')


print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

mode = 0o666

if localEnv is True:
    inputZipFolderPath = inputZipFolderPath.replace("/","\\")
    outputFolderPath = outputFolderPath.replace("/","\\")
    controlFolderPath = controlFolderPath.replace("/","\\")

try:
    os.makedirs(inputZipFolderPath, mode)
    os.makedirs(outputFolderPath, mode)
    os.makedirs(controlFolderPath, mode)

except Exception:
    print("Already Present")
    
with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
        zip_ref.extractall(outputFolderPath)
    

_,_,fileNames = next(os.walk(outputFolderPath))
htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

print(htmlFileName)



Zynteglo~H~CAP~en~2021-05-19T07-52-54Z.zip F:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z F:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Zynteglo_clean.htm


In [17]:
a,b,c = parseDocument(controlFolderPath, outputFolderPath, htmlFileName, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName)

2021-05-18 23:30:47,797 : Flow Logger HTML_n : Starting HTML Conversion To Json | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:47,803 : Style Dictionary_v : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:47,832 : Style Dictionary_v : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | HTML | Zynteglo_clean.htm


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'Zynteglo', 'en', '2021-05-19T07-52-54Z'] Zynteglo_clean.htm
2021-05-19T07-52-54Z en Zynteglo CAP H
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\Zynteglo_clean.htm F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.json


2021-05-18 23:30:48,143 : Parser_D : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.txt | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,675 : Parser_D : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.json | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,769 : Flow Logger HTML_n : Completed HTML Conversion To Json | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,769 : Flow Logger HTML_n : Starting Json Split | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,777 : Style Dictionary_V : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Zynteglo_clean.json
2021-05-18 23:30:49,804 : Style Dictionary_V : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET |

stylePath:- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.txt
PathJson F:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Zynteglo\en\2021-05-19T07-52-54Z\outputJSON\Zynteglo_clean.json


2021-05-18 23:30:49,956 : Flow Logger HTML_n : ['Zynteglo_clean_SmPC.json', 'Zynteglo_clean_ANNEX II.json', 'Zynteglo_clean_ANNEX III.json', 'Zynteglo_clean_ PACKAGE LEAFLET.json'] | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,956 : Flow Logger HTML_n : Completed Json Split | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,956 : Flow Logger HTML_n : Started Processing Partitioned Jsons | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,956 : Flow Logger HTML_n : 



||||||||||||||||||||||||||||||||0 ||||| Zynteglo_clean_SmPC.json||||||||||||||||||||||||||||||||



 | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:30:49,964 : Heading Extraction Zynteglo_clean_SmPC.json_B : Starting Heading Extraction | H | CAP |  en | 0 | Zynteglo_clean_SmPC.json
2021-05-18 23:30:50,084 : Heading Extraction Zynteglo_clean_SmPC.json_B : Started Extracting Heading | H | CAP |  en | 0 | Zynteglo_clean_SmPC.json


...

2021-05-18 23:32:19,518 : Flow Logger HTML_n : Completed Heading Extraction For File | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:32:19,518 : Flow Logger HTML_n : Starting Document Annotation For File | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:32:19,526 : Flow Logger HTML_n : Completed Document Annotation | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:32:19,526 : Flow Logger HTML_n : Starting Extracting Content Between Heading | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:32:19,526 : ExtractContentBetween_3_l : Cleaning Match Results | H | CAP |  en | 3 | Zynteglo_clean_ PACKAGE LEAFLET.json
2021-05-18 23:32:19,534 : ExtractContentBetween_3_l : Finished Cleaning Match Results | H | CAP |  en | 3 | Zynteglo_clean_ PACKAGE LEAFLET.json
2021-05-18 23:32:19,550 : Flow Logger HTML_n : Completed Extracting Content Between Heading | H | CAP |  en | HTML | Zynteglo_clean.htm
2021-05-18 23:32:19,566 : XmlGeneration_3_e : PMS/OMS Annotation Info



Heading Not Found 
 ['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the end of section 4 for how to report side effects.', 'Do not <take> <use> X', 'Children <and adolescents>', 'X with <food> <and> <,> <drink> <and> <alcohol>', 'X contains {name the excipient(s)}', 'How to <take> <use> X ', 'Use in children <and adolescents>', 'If you <take> <use> more X than you should', 'If you forget to <take> <use> X>', 'If you stop <taking> <using> X>', 'Additional side effects in children <and adolescents>', 'This leaflet was last revised in <{MM/YYYY}><{month YYYY}>.', 'Other sources of information']


dict_keys(['q This medicine is subject to additional monitoring. This will allow quick identification of new safety information. You can help by reporting any side effects you may get. See the end of section 4 for how to report side effects.', '1. What Zynteglo is a

2021-05-18 23:32:22,310 : XML Submission Logger_3_t : Initiating Submission To FHIR Server | H | CAP |  en | 3 | Zynteglo_clean_ PACKAGE LEAFLET.json
2021-05-18 23:32:22,314 : XML Submission Logger_3_t : Response{"resourceType":"Bundle","id":"44e0ecad-f1a2-4339-a584-55ad47f2c601","meta":{"versionId":"1","lastUpdated":"2021-05-18T18:02:21.681+00:00"},"type":"collection","entry":[{"fullUrl":"urn:uuid:1dac9f6e-0f09-4246-a421-a42c2247f7d6","resource":{"resourceType":"Bundle","id":"dd13ccb5-13c9-41a9-9156-2eb20d813ae8","identifier":{"system":"http://ema.europa.eu/fhir/identifier/documentid","value":"${instance.bundle[n].Identifier}"},"type":"document","timestamp":"2021-05-18T18:02:19+00:00","entry":[{"fullUr | H | CAP |  en | 3 | Zynteglo_clean_ PACKAGE LEAFLET.json
2021-05-18 23:32:22,326 : XML Submission Logger_3_t : POST sucessful: XML added with id: 44e0ecad-f1a2-4339-a584-55ad47f2c601 | H | CAP |  en | 3 | Zynteglo_clean_ PACKAGE LEAFLET.json
2021-05-18 23:32:22,326 : Flow Logger HTML_

POST sucessful: XML added with id 44e0ecad-f1a2-4339-a584-55ad47f2c601
Created XML File For :- Zynteglo_clean_ PACKAGE LEAFLET.json


TypeError: cannot unpack non-iterable NoneType object

In [9]:
a

,Bold,Classes,Element,HasBorder,ID,Indexed,IsHeadingType,IsListItem,IsPossibleHeading,Italics,ParentId,Styles,Text,Underlined,Uppercased,StringLength
0,False,['WordSection1'],"<div class=""WordSection1""> <p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p> <p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p> <p class=""MsoNormal"" style=...",False,8fc90752-ec33-4215-99a3-8711ebe75633,False,None,False,False,False,55adea44-12a5-4e30-801d-307fadde0f35,None,...,False,False,0
1,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,a3e0a75e-ace0-4f81-88d8-03ae112aecc9,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
2,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,d9317af2-2971-4d70-a655-a7a31caf7c20,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
3,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,f43032db-d077-4fec-a526-0483819905e4,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
4,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,39eb0909-d6d3-4461-8ef2-6efc5007a7d9,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
5,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,01a2b620-5f4a-4590-a08c-7566a4e5fd62,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
6,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,1d22d9d3-79f7-4b34-86d0-6a439d7e9f30,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
7,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,8a80fed1-b3f1-4ed9-bf09-9df7f7e75280,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
8,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,3450fbac-4745-43b1-baa3-59c7bb10378e,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0
9,False,['MsoNormal'],"<p class=""MsoNormal"" style=""margin-bottom:0in;line-height:normal""></p>",False,5ca680b8-2583-4976-aa18-46996e2fce99,False,None,False,False,False,8fc90752-ec33-4215-99a3-8711ebe75633,margin-bottom:0in;line-height:normal,,False,False,0


In [14]:
convertCollectionToDataFrame(b)

,index,id,Procedure type,Display code,Name,parent_id,htmlText,htmlIndex,htmlId,SubSectionIndex,doc_parent_id
0,680,20001,CAP,NaN,SUMMARY OF PRODUCT CHARACTERISTICS,NaN,SUMMARY\r OF PRODUCT CHARACTERISTICS,26,7b37353a-5e3e-400a-9acf-6b44224c82e4,0,
1,682,20003,CAP,1,NAME OF THE MEDICINAL PRODUCT,20001,1. NAME OF THE\r MEDICINAL PRODUCT,33,62ecb297-5186-4c91-8144-6540859f879b,0,20001
2,683,20004,CAP,2,QUALITATIVE AND QUANTITATIVE COMPOSITION,20001,2. QUALITATIVE AND\r QUANTITATIVE COMPOSITION,38,5003c211-0a0d-4c38-8258-71d672ed3498,0,20001
3,684,20005,CAP,2.1,General description,20004,2.1 General description,40,ae7ffbb8-396b-4813-95f5-9f3eb67bcfa3,0,20004
4,685,20006,CAP,2.2,Qualitative and quantitative composition,20004,2.2 Qualitative and\r quantitative composition,44,054b75a3-b7a0-4877-b8f4-65cd9f0b5bc7,0,20004
5,686,20007,CAP,NaN,Excipient(s) with known effect,20006,Excipient with known effect,50,708fa882-ab8b-49b7-8745-a2757401ec2f,0,20006
6,687,20008,CAP,3,PHARMACEUTICAL FORM,20001,3. PHARMACEUTICAL\r FORM,57,cd25c04e-2fec-477f-a67d-b894d46f03c9,0,20001
7,688,20009,CAP,4,CLINICAL PARTICULARS,20001,4. CLINICAL\r PARTICULARS,64,12a07622-b5ed-483d-bda8-96814b599bc9,0,20001
8,689,20010,CAP,4.1,Therapeutic indications,20009,4.1 Therapeutic\r indication,66,4dcf34cc-26ef-4720-87fa-435048832ef7,0,20009
9,690,20011,CAP,4.2,Posology and method of administration,20009,4.2 Posology and\r method of administration,70,68b9ace8-fc3c-429f-92b8-e983e9d1e388,0,20009


In [13]:
def convertToInt(x):
    try:
        return str(int(x))
    except:
        return x


def convertCollectionToDataFrame(collection):

    dfExtractedHier = pd.DataFrame(collection)
    dfExtractedHier['parent_id'] = dfExtractedHier['parent_id'].apply(
        lambda x: convertToInt(x))
    dfExtractedHier['id'] = dfExtractedHier['id'].apply(
        lambda x: convertToInt(x))

    return dfExtractedHier

In [7]:
import jellyfish

In [22]:
jaroWinklerScore = jellyfish.jaro_winkler_similarity('2.2 qualitative and quantitative composition'.title(), '2. qualitative and quantitative composition'.title())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
str(round(jaroWinklerScore, 3))

'0.994'